# KV Data Cleanup

## Starting up

In [1]:
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("kv-rent-data-16-11-2024.csv")

In [4]:
# underscores are easier to work with than hyphens.
data.columns = data.columns.str.replace('-', '_')
# As we can see, the situation is pretty bad by default.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2676 entries, 0 to 2675
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   web_scraper_order       2676 non-null   object 
 1   web_scraper_start_url   2676 non-null   object 
 2   listing_link            2676 non-null   object 
 3   listing_link_href       2676 non-null   object 
 4   address                 2676 non-null   object 
 5   price                   2676 non-null   object 
 6   rooms                   2667 non-null   float64
 7   area                    2670 non-null   object 
 8   floor_out_of_floors     2494 non-null   object 
 9   build_year              2081 non-null   float64
 10  condition               2456 non-null   object 
 11  energy_grade            2311 non-null   object 
 12  summary                 2675 non-null   object 
 13  description             2633 non-null   object 
 14  bedrooms                1848 non-null   

In [5]:
# Let's remove the Web Scraper columns.
# listing-link and address have the same info, but address has more bloat. 
# listing-link is easier to filter and split. let's remove address as well
data = data.drop(['web_scraper_order', 'web_scraper_start_url', 'address'], axis=1)

## floor_out_of_floors deserves its own chapter - cell formatting is annoying

In [6]:
# floor_out_of_floors seems to have had a bit of a mishap. 
# they were automatically transformed to a date, but they were actually 1/5, 3/5, 1/2 etc.
# let's separate all the columns that are easily separable - price, floor_out_of_floors
# first, let's check the unique values.
data['floor_out_of_floors'].unique()
# most of them are pretty clear - Day-Month corresponds to FLOOR-TOTAL_FLOORS
# some, however aren't clear: -0.25, -0.333333333, -0.2. These need to be checked individually.

array(['01-Mar', '03-May', '04-Apr', '01-Feb', '03-Apr', '01-Apr',
       '02-Mar', '05-May', '01-May', '04-May', '03-Mar', nan, '02-May',
       '06-Jun', '03-Jun', '03-Jul', '04-Feb', '05-Jun', '02-Apr',
       '02-Feb', '07-Sep', '04-Jun', '04-Sep', '06-Aug', '09-Sep',
       '03-Aug', '04-Jul', '02-Jun', '08-Aug', '05-Jul', '01-Jun',
       '01-Aug', '08-Dec', 'Dec-14', '03-Sep', '06-Jul', '01-Jan',
       '08-Sep', '04-Aug', '13/30', 'May-14', '01-Sep', 'Apr-14',
       '05-Oct', '18/23', '06-Sep', '15/30', '07-Aug', '05-Aug', 'Aug-15',
       '05-Sep', '-0.25', '04-Oct', '02-Jan', '02-Jul', '10-Nov',
       '02-Sep', 'Jul-16', 'Jun-14', 'Oct-19', 'Jun-13', '09-Dec',
       'Feb-14', '10-Dec', '24/30', '15/16', 'Dec-13', '02-Aug', 'May-13',
       '07-Nov', '14/20', 'Sep-14', 'Oct-17', '07-Jul', '09-Oct',
       '03-Oct', '08-Oct', '03-Feb', '06-Dec', 'Jul-20', 'Nov-19',
       'Jul-19', 'Jul-14', '14/14', 'Apr-22', '09-Nov', 'Jun-16',
       'Dec-16', 'Apr-13', 'Dec-15', 'Aug-19'

In [7]:
#data[data['floor_out_of_floors'] == '-0.25'].iloc[0]['listing-link-href'] # -0.25 is actually a basement floor -1/4
#data[data['floor_out_of_floors'] == '-0.25'].iloc[1]['listing-link-href'] # -0.25 is actually a basement floor -1/4
# ^ those two listings are actually the same apartment listing two times, 
# the links really are different, though.
#data[data['floor_out_of_floors'] == '-0.333333333'].iloc[0]['listing-link-href'] # -0.333333333 is actually a basement floor -1/3
#data[data['floor_out_of_floors'] == '-0.2'].iloc[0]['listing-link-href'] # -0.2 is actually a basement floor -1/5 

In [8]:
# The plan is the following (not the most optimal, but definitely won't break anything):
# map -0.25, -0.333333333, -0.2 to -1/4, -1/3, -1/5
# map [Jan, Feb, Mar,...] to [1,2,3,...] in each string
# somehow this needs to apply to substrings. 
# map - to / DANGER, map only once FROM RIGHT, otherwise negative floor numbers will be affected
# check all unique values
# if unique values are all good, then split from / and cast to int
## The following is created with help from Claude.ai.
def transform_floor_numbers(value):
    if pd.isna(value):
        return value
        
    # First handle the special basement cases
    basement_map = {
        '-0.25': '-1/4',
        '-0.333333333': '-1/3',
        '-0.2': '-1/5'
    }
    if str(value) in basement_map:
        return basement_map[str(value)]
    
    # Handle month name conversions
    month_map = {
        'Jan': '1', 'Feb': '2', 'Mar': '3', 'Apr': '4', 
        'May': '5', 'Jun': '6', 'Jul': '7', 'Aug': '8', 
        'Sep': '9', 'Oct': '10', 'Nov': '11', 'Dec': '12'
    }
    
    value = str(value)
    # Handle cases like "Dec-14"
    for month, num in month_map.items():
        if month in value:
            value = value.replace(month, num)
    
    # Handle cases where - needs to be converted to / (but only rightmost occurrence)
    if '-' in value and '/' not in value:
        parts = value.rsplit('-', 1)  # Split from right once
        value = parts[0] + '/' + parts[1]
        
    return value

# Apply the transformation
data['floor_out_of_floors'] = data['floor_out_of_floors'].apply(transform_floor_numbers)

In [9]:
data['floor_out_of_floors'].unique()

array(['01/3', '03/5', '04/4', '01/2', '03/4', '01/4', '02/3', '05/5',
       '01/5', '04/5', '03/3', nan, '02/5', '06/6', '03/6', '03/7',
       '04/2', '05/6', '02/4', '02/2', '07/9', '04/6', '04/9', '06/8',
       '09/9', '03/8', '04/7', '02/6', '08/8', '05/7', '01/6', '01/8',
       '08/12', '12/14', '03/9', '06/7', '01/1', '08/9', '04/8', '13/30',
       '5/14', '01/9', '4/14', '05/10', '18/23', '06/9', '15/30', '07/8',
       '05/8', '8/15', '05/9', '-1/4', '04/10', '02/1', '02/7', '10/11',
       '02/9', '7/16', '6/14', '10/19', '6/13', '09/12', '2/14', '10/12',
       '24/30', '15/16', '12/13', '02/8', '5/13', '07/11', '14/20',
       '9/14', '10/17', '07/7', '09/10', '03/10', '08/10', '03/2',
       '06/12', '7/20', '11/19', '7/19', '7/14', '14/14', '4/22', '09/11',
       '6/16', '12/16', '4/13', '12/15', '8/19', '04/11', '07/10', '-1/3',
       '11/14', '14/16', '02/12', '10/10', '11/13', '05/12', '3/16',
       '5/16', '17/17', '17/20', '02/10', '03/12', '11/15', '09/4',
  

In [10]:
data[['floor', 'total_floors']] = data['floor_out_of_floors'].str.split('/', expand=True)
data['floor'] = pd.to_numeric(data['floor'], errors='coerce')
data['total_floors'] = pd.to_numeric(data['total_floors'], errors='coerce')

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2676 entries, 0 to 2675
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   listing_link            2676 non-null   object 
 1   listing_link_href       2676 non-null   object 
 2   price                   2676 non-null   object 
 3   rooms                   2667 non-null   float64
 4   area                    2670 non-null   object 
 5   floor_out_of_floors     2494 non-null   object 
 6   build_year              2081 non-null   float64
 7   condition               2456 non-null   object 
 8   energy_grade            2311 non-null   object 
 9   summary                 2675 non-null   object 
 10  description             2633 non-null   object 
 11  bedrooms                1848 non-null   float64
 12  ownership_form          1923 non-null   object 
 13  katastrinumber          1710 non-null   object 
 14  description_header      2282 non-null   

## Elementary column transformations - extracting and cleaning up easy values

In [12]:
# price -> price, price_per_m2
data[['price', 'price_per_m2']] = data['price'].str.split('€ ', n=1, expand=True)
data['price'] = pd.to_numeric(data['price'].str.strip(), errors='coerce')
data['price_per_m2'] = data['price_per_m2'].str.replace('€/m²', '').str.strip()
data['price_per_m2'] = pd.to_numeric(data['price_per_m2'], errors='coerce')

In [13]:
# area -> float64 area with filtering
data['area'] = data['area'].str.replace('\xa0m²', '').str.strip()
data['area'] = pd.to_numeric(data['area'], errors='coerce')

In [14]:
# energy_grade to numeric, A is highest, H is lowest value
data.energy_grade = data.energy_grade.map({
    'Puudub': np.nan, '-': np.nan,
    'H':1, 'G':2, 'F':3, 'E':4, 'D':5, 'C':6, 'B':7, 'A':8
})

In [15]:
# arbitrary condition mapping to 5 numeric categories
data['condition'] = data['condition'].map({
    'Uus': 5, 'Uusarendus': 5,
    'Renoveeritud': 4,
    'Valmis': 3, 'Heas korras': 3,
    'San. remont tehtud': 2, 'Keskmine': 2,
    'Vajab san. remonti': 1, 'Vajab renoveerimist': 1
})


In [16]:
# arbitrary ownership_form mapping to numeric categories
data['ownership_form'] = data['ownership_form'].map({
    'Korteriomand': 5, 'Kinnistu': 5,
    'Elamuühistu': 4,
    'Kaasomand': 3, 'Hoonestusõigus': 3,
    'Üürileping': 2,
    'Vallasasi': 1
})

In [17]:
# get copy_not_allowed and broker_not_allowed from the footer
# used Copilot to simplify this
def map_description_footer(df):
    df['copy_not_allowed'] = df['description_footer'].apply(lambda x: 'Ei luba enda kuulutust kopeerida' in x)
    df['no_broker_allowed'] = df['description_footer'].apply(lambda x: 'Maakleritel palun mitte tülitada' in x)
    return df

data = map_description_footer(data)

In [18]:
# from images_link, we can get the number of pictures attached to the post. that seems like a worthwhile data point to have
# we can use a regex function for that. used Copilot to check the correct regex function in python
def extract_images_attached(df):
    df['images_attached'] = df['images_link'].str.extract(r'\((\d+)\)')
    df['images_attached'] = pd.to_numeric(df['images_attached'], errors='coerce')
    return df

data = extract_images_attached(data)

In [19]:
# let's have all the column dropping in the last cell.
data = data.drop(['images_link', 'description_footer', 'floor_out_of_floors'], axis=1)

data.info()

## Removing some extreme outliers

In [20]:
# two rows have 29 rooms listed.
# those are actually "shared housing" listing & no rooms cost 200 - the cheapest is 350. Let's drop those rows.
data = data.drop(data[data.rooms == 29].index)

In [21]:
#data[data.rooms.isna()] # 9 rows, let's leave those in for "comparison with no room nr"
#data[data.rooms == 8] # 2 rows, seem correct
#data[data.rooms == 6] # 5 rows
#data[data.rooms == 5] # 22 rows, that's definitely good enough

In [22]:
# row with index 2145 has price 1€ and area as 1 m2. Not going to bother finding out what's happening.
data = data.drop(2145)

## Extracting address from listing_link

In [23]:
# TODO

## Create a new column with coordinates from address using external library

In [25]:
# TODO

## Extracting info from summary                 

In [26]:
# TODO

## Extracting info from description

In [27]:
# TODO

## Determine whether 'katastrinumber' has any value to us after we already have the address - what even is it?

In [28]:
# TODO

## Extract info from description header - is it in all caps (bool)? Count !? and something like that maybe?

In [29]:
# TODO

## MOVE TO ELEMENTARY COLUMN TRANSFORMATIONS: Transform column prepayment to float64 (mostly useful for NaN or no NaN)

In [30]:
# TODO

## MOVE TO ELEMENTARY COLUMN TRANSFORMATIONS:  Extract info from owner_or_broker_banner - BOOL: is the listing created by the owner? true -> owner,  false -> broker

In [31]:
# TODO

## MOVE TO ELEMENTARY COLUMN TRANSFORMATIONS:  Determine whether 'registriosa_number' has any value to us

In [32]:
# TODO

## Determine whether to try to analyse pictures from images_link_href and assign a rating to those

In [33]:
# TODO